In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import torch
import random
print(torch.cuda.is_available())

True


In [2]:
#ファイル入力
def file_input(input_path,output_path):
    with open(input_path) as f:
        s = f.read()
    s = s.strip().replace('\t','\n').replace('.','')
    sentences = s.split('\n')
    
    ps = PorterStemmer()
    for i in range(len(sentences)):
        token = sentences[i].split(' ')
        singles = [ps.stem(plural) for plural in token]
        sentences[i] = singles
    return sentences

In [3]:
# 単語IDの登録
def word_to_id(sentences:list):
    word_id_list = {}
    id_count = 0
    for sentence in sentences:
        for token in sentence:
            if(token not in word_id_list):        
                word_id_list[token] = id_count
                id_count += 1
    return word_id_list

In [4]:
#単語からワンホットベクトル列を入手。
def word_to_vector(word:str,dim,word_id_list:dict):
    X = torch.zeros(dim)
    X[word_id_list[word]] = 1.0
    return X
# word_to_vector('young',dim,word_id_list)
# print(word_id_list['young'])


In [5]:
def get_n_gram(sentences:list):
    ngrams = []
    for sentence in sentences:
        l = len(sentence)
        for i in range(l-2):
            list = [sentence[i],sentence[i+2]]
            ngrams.append((list,sentence[i+1]))
    return ngrams

In [6]:
class CBOW(torch.nn.Module):
    def __init__(self,vocab_size,embedding_dim,context_size):
        super(CBOW,self).__init__()
        self.embeddings = torch.nn.Embedding(vocab_size,embedding_dim)
        self.linear1 = torch.nn.Linear(context_size*embedding_dim,128)
        self.linear2 = torch.nn.Linear(128,vocab_size)

    def forward(self,inputs):
        embeds = self.embeddings(inputs).view(1,-1)
        out = torch.nn.functional.relu(self.linear1(embeds))
        out = self.linear2(out)
        return out

In [7]:

input_path = '../test_data/STS.input.images.txt'
#input_path = '../test_data/testinput.txt'
output_path = './output/STS.output.images.txt'
#output_path = './output/testoutput.txt'
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

#ファイル入力
sentences = file_input(input_path,output_path)
# sentences = sentences[0:2]
# print(sentences)

word_to_ix = word_to_id(sentences)
#print(word_id_list)
vocab_size = len(word_to_ix)

ngrams = get_n_gram(sentences)
#print(ngrams)

# x,y = create_contexts_target(sentences)
# random.shuffle(x)
# random.shuffle(y)
# vocab_size = len(x)
# print(vocab_size)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fox25\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# #device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# #torch.set_default_tensor_type('torch.cuda.FloatTensor')
# CONTEXT_SIZE = 2
# EMBEDDING_DIM = 10

# losses = []
# loss_function = torch.nn.CrossEntropyLoss()
# model = CBOW(vocab_size,EMBEDDING_DIM,CONTEXT_SIZE)
# optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
# for epoch in range(400):
#     total_loss = torch.Tensor([0])
#     for context,target in ngrams:
#         context_idxs = [word_to_ix[w] for w in context]
#         # print(context_idxs)
#         context_var = torch.autograd.Variable(torch.LongTensor(context_idxs))
#         # print(context_var)
#         model.zero_grad()
#         output = model(context_var)
#         loss = loss_function(output,torch.autograd.Variable(torch.LongTensor([word_to_ix[target]])))
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.data
#     losses.append(total_loss.detach().numpy()[0])
#     print(epoch,total_loss)

In [46]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_default_tensor_type('torch.cuda.FloatTensor')
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

losses = []
loss_function = torch.nn.CrossEntropyLoss()
model = CBOW(vocab_size,EMBEDDING_DIM,CONTEXT_SIZE).to('cuda')
#model.load_state_dict(torch.load(PATH))
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
for epoch in range(400):
    total_loss = torch.Tensor([0])
    for context,target in ngrams:
        context_idxs = [word_to_ix[w] for w in context]
        # print(context_idxs)
        # context_var = torch.autograd.Variable(torch.LongTensor(context_idxs)).to(device)
        context_var = torch.autograd.Variable(torch.LongTensor(context_idxs)).to('cuda')
        # print(context_var)
        model.zero_grad()
        #output = model(context_var)
        output = model(context_var)
        loss = loss_function(output,torch.autograd.Variable(torch.LongTensor([word_to_ix[target]])).to('cuda'))
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    #losses.append(total_loss.cpu().detach().numpy()[0])
    total_loss.to('cpu')
    losses.append(total_loss.item())
    #losses.append(total_loss.detach().numpy()[0])
    print(epoch,total_loss.item())
    PATH = './output/' + f'model_{epoch}'
    torch.save(model.state_dict(),PATH)

0 61149.94140625


KeyboardInterrupt: 

In [48]:
state_dict = model.state_dict()
print(state_dict.keys())
w = state_dict['embeddings.weight']
w.size()
print(w.size())
print(vocab_size)

odict_keys(['embeddings.weight', 'linear1.weight', 'linear1.bias', 'linear2.weight', 'linear2.bias'])
torch.Size([981, 10])
981


In [12]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import numpy as np
# import urllib.request
# from nltk.tokenize import RegexpTokenizer
# from nltk.corpus import stopwords
# from nltk import word_tokenize
# import sklearn
# from sklearn.cluster import KMeans
# from sklearn.metrics.pairwise import euclidean_distances

# torch.manual_seed(1)

# CONTEXT_SIZE = 3
# EMBEDDING_DIM = 10

# test_sentence = """Empathy for the poor may not come easily to people who never experienced it. They may blame the victims and insist their predicament can be overcome through determination and hard work.
# But they may not realize that extreme poverty can be psychologically and physically incapacitating — a perpetual cycle of bad diets, health care and education exacerbated by the shaming and self-fulfilling prophecies that define it in the public imagination.
# Gordon Parks — perhaps more than any artist — saw poverty as “the most savage of all human afflictions” and realized the power of empathy to help us understand it. It was neither an abstract problem nor political symbol, but something he endured growing up destitute in rural Kansas and having spent years documenting poverty throughout the world, including the United States.
# That sensitivity informed “Freedom’s Fearful Foe: Poverty,” his celebrated photo essay published in Life magazine in June 1961. He took readers into the lives of a Brazilian boy, Flavio da Silva, and his family, who lived in the ramshackle Catacumba favela in the hills outside Rio de Janeiro. These stark photographs are the subject of a new book, “Gordon Parks: The Flavio Story” (Steidl/The Gordon Parks Foundation), which accompanies a traveling exhibition co-organized by the Ryerson Image Centre in Toronto, where it opens this week, and the J. Paul Getty Museum. Edited with texts by the exhibition’s co-curators, Paul Roth and Amanda Maddox, the book also includes a recent interview with Mr. da Silva and essays by Beatriz Jaguaribe, Maria Alice Rezende de Carvalho and Sérgio Burgi.
# """.split()
# # we should tokenize the input, but we will ignore that for now
# # build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

# #print(test_sentence)
# print(len(test_sentence))
# def get_key(word_id):
#     for key,val in word_to_ix.items():
#         if(val == word_id):
#             print(key)

# def cluster_embeddings(filename,nclusters):
#     X = np.load(filename)
#     kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(X)
#     center = kmeans.cluster_centers_
#     distances = euclidean_distances(X,center)

#     for i in np.arange(0,distances.shape[1]):
#         word_id = np.argmin(distances[:,i])
#         print(word_id)
#         get_key(word_id)

# def read_data(file_path):
#     tokenizer = RegexpTokenizer(r'\w+')
#     data = urllib.request.urlopen(file_path)
#     data = data.read().decode('utf8')
#     tokenized_data = word_tokenize(data)
#     stop_words = set(stopwords.words('english'))
#     stop_words.update(['.',',',':',';','(',')','#','--','...','"'])
#     cleaned_words = [ i for i in tokenized_data if i not in stop_words ]
#     return(cleaned_words)

# #test_sentence = read_data('https://www.gutenberg.org/files/57884/57884-0.txt')

# ngrams = []
# for i in range(len(test_sentence) - CONTEXT_SIZE):
#     tup = [test_sentence[j] for j in np.arange(i , i + CONTEXT_SIZE) ]
#     ngrams.append((tup,test_sentence[i + CONTEXT_SIZE]))
# # print the first 3, just so you can see what they look like
# print(ngrams)


# vocab = set(test_sentence)
# print(vocab)
# print("Length of vocabulary",len(vocab))
# word_to_ix = {word: i for i, word in enumerate(vocab)}
# print(word_to_ix)


# class CBOWModeler(nn.Module):

#     def __init__(self, vocab_size, embedding_dim, context_size):
#         super(CBOWModeler, self).__init__()
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         self.linear1 = nn.Linear(context_size * embedding_dim, 128)
#         self.linear2 = nn.Linear(128, vocab_size)

#     def forward(self, inputs):
#         embeds = self.embeddings(inputs).view((1, -1))  # -1 implies size inferred for that index from the size of the data
#         #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
#         out1 = F.relu(self.linear1(embeds)) # output of first layer
#         out2 = self.linear2(out1)           # output of second layer
#         #print(embeds)
#         log_probs = F.log_softmax(out2, dim=1)
#         return log_probs

#     def predict(self,input):
#         context_idxs = torch.tensor([word_to_ix[w] for w in input], dtype=torch.long)
#         res = self.forward(context_idxs)
#         res_arg = torch.argmax(res)
#         res_val, res_ind = res.sort(descending=True)
#         res_val = res_val[0][:3]
#         res_ind = res_ind[0][:3]
#         #print(res_val)
#         #print(res_ind)
#         for arg in zip(res_val,res_ind):
#             #print(arg)
#             print([(key,val,arg[0]) for key,val in word_to_ix.items() if val == arg[1]])

#     def freeze_layer(self,layer):
#         for name,child in model.named_children():
#             print(name,child)
#             if(name == layer):
#                 for names,params in child.named_parameters():
#                     print(names,params)
#                     print(params.size())
#                     params.requires_grad= False

#     def print_layer_parameters(self):
#         for name,child in model.named_children():
#                 print(name,child)
#                 for names,params in child.named_parameters():
#                     print(names,params)
#                     print(params.size())

#     def write_embedding_to_file(self,filename):
#         for i in self.embeddings.parameters():
#             weights = i.data.numpy()
#         np.save(filename,weights)



# losses = []
# loss_function = nn.NLLLoss()
# model = CBOWModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
# optimizer = optim.SGD(model.parameters(), lr=0.001)

# # Freeze embedding layer
# #model.freeze_layer('embeddings')


# for epoch in range(400):
#     total_loss = 0
#     #------- Embedding layers are trained as well here ----#
#     #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
#     #hello_embed = model.embeddings(lookup_tensor)
#     #print(hello_embed)
#     # -----------------------------------------------------#

#     for context, target in ngrams:

#         # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
#         # into integer indices and wrap them in tensors)
#         context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
#         #print("Context id",context_idxs)

#         # Step 2. Recall that torch *accumulates* gradients. Before passing in a
#         # new instance, you need to zero out the gradients from the old
#         # instance
#         model.zero_grad()

#         # Step 3. Run the forward pass, getting log probabilities over next
#         # words
#         print(epoch)
#         print(context_idxs)
#         log_probs = model(context_idxs)
#         #print(log_probs)

#         # Step 4. Compute your loss function. (Again, Torch wants the target
#         # word wrapped in a tensor)
#         loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
#         #print(loss)

#         # Step 5. Do the backward pass and update the gradient
#         loss.backward()
#         optimizer.step()

#         # Get the Python number from a 1-element Tensor by calling tensor.item()
#         total_loss += loss.item()
#     print(total_loss)
#     losses.append(total_loss)
# #print(losses)  # The loss decreased every iteration over the training data!

# #Print the model layer parameters
# #model.print_layer_parameters()

# #Predict the next word given n context words
# model.predict(['of','all','human'])
# model.write_embedding_to_file('embeddings.npy')
# cluster_embeddings('embeddings.npy',2)

259
[(['Empathy', 'for', 'the'], 'poor'), (['for', 'the', 'poor'], 'may'), (['the', 'poor', 'may'], 'not'), (['poor', 'may', 'not'], 'come'), (['may', 'not', 'come'], 'easily'), (['not', 'come', 'easily'], 'to'), (['come', 'easily', 'to'], 'people'), (['easily', 'to', 'people'], 'who'), (['to', 'people', 'who'], 'never'), (['people', 'who', 'never'], 'experienced'), (['who', 'never', 'experienced'], 'it.'), (['never', 'experienced', 'it.'], 'They'), (['experienced', 'it.', 'They'], 'may'), (['it.', 'They', 'may'], 'blame'), (['They', 'may', 'blame'], 'the'), (['may', 'blame', 'the'], 'victims'), (['blame', 'the', 'victims'], 'and'), (['the', 'victims', 'and'], 'insist'), (['victims', 'and', 'insist'], 'their'), (['and', 'insist', 'their'], 'predicament'), (['insist', 'their', 'predicament'], 'can'), (['their', 'predicament', 'can'], 'be'), (['predicament', 'can', 'be'], 'overcome'), (['can', 'be', 'overcome'], 'through'), (['be', 'overcome', 'through'], 'determination'), (['overcome', 

KeyboardInterrupt: 

5
tensor([180,  12,  40])
5
tensor([12, 40, 27])
5
tensor([40, 27, 44])
5
tensor([ 27,  44, 137])
5
tensor([ 44, 137,  47])
5
tensor([137,  47, 183])
5
tensor([ 47, 183,  41])
5
tensor([183,  41,  25])
5
tensor([ 41,  25, 184])
5
tensor([ 25, 184,  40])
5
tensor([184,  40,  17])
5
tensor([40, 17, 11])
5
tensor([ 17,  11, 143])
5
tensor([ 11, 143, 138])
5
tensor([143, 138,  80])
5
tensor([138,  80,  66])
5
tensor([ 80,  66, 154])
5
tensor([ 66, 154,  48])
5
tensor([154,  48,  84])
5
tensor([ 48,  84, 186])
5
tensor([ 84, 186,  47])
5
tensor([186,  47, 183])
5
tensor([ 47, 183, 178])
5
tensor([183, 178,  40])
5
tensor([178,  40, 153])
5
tensor([ 40, 153, 132])
5
tensor([153, 132,  86])
5
tensor([132,  86, 117])
5
tensor([ 86, 117,  37])
5
tensor([117,  37,  13])
5
tensor([37, 13, 49])
5
tensor([ 13,  49, 142])
5
tensor([ 49, 142, 193])
5
tensor([142, 193, 102])
5
tensor([193, 102, 181])
5
tensor([102, 181,  40])
5
tensor([181,  40,   1])
5
tensor([40,  1,  4])
5
tensor([ 1,  4, 35])
5
te

6
tensor([148, 135, 174])
6
tensor([135, 174,  67])
6
tensor([174,  67,  81])
6
tensor([ 67,  81, 185])
6
tensor([ 81, 185,  31])
6
tensor([185,  31,  19])
6
tensor([ 31,  19, 149])
6
tensor([ 19, 149, 167])
6
tensor([149, 167, 124])
6
tensor([167, 124,  26])
6
tensor([124,  26, 159])
6
tensor([ 26, 159,  96])
6
tensor([159,  96, 162])
6
tensor([ 96, 162,  61])
6
tensor([162,  61,  96])
6
tensor([ 61,  96, 121])
6
tensor([ 96, 121,  83])
6
tensor([121,  83, 190])
6
tensor([ 83, 190, 110])
6
tensor([190, 110,  58])
6
tensor([110,  58, 101])
6
tensor([ 58, 101, 180])
6
tensor([101, 180, 103])
6
tensor([180, 103,  49])
6
tensor([103,  49, 117])
6
tensor([ 49, 117,  46])
6
tensor([117,  46,  59])
6
tensor([46, 59, 76])
6
tensor([59, 76,  3])
6
tensor([ 76,   3, 146])
6
tensor([  3, 146,  40])
6
tensor([146,  40, 149])
6
tensor([ 40, 149, 118])
6
tensor([149, 118,  85])
6
tensor([118,  85,   5])
6
tensor([85,  5, 96])
6
tensor([  5,  96, 180])
6
tensor([ 96, 180, 158])
6
tensor([180, 158, 1

7
tensor([22, 33, 95])
7
tensor([ 33,  95, 172])
7
tensor([ 95, 172, 177])
7
tensor([172, 177,   6])
7
tensor([177,   6,  40])
7
tensor([  6,  40, 164])
1304.6304860115051
8
tensor([ 92,  69, 180])
8
tensor([ 69, 180, 179])
8
tensor([180, 179,  80])
8
tensor([179,  80,  66])
8
tensor([ 80,  66, 141])
8
tensor([ 66, 141, 163])
8
tensor([141, 163,  72])
8
tensor([163,  72,  29])
8
tensor([72, 29, 85])
8
tensor([29, 85, 97])
8
tensor([85, 97, 24])
8
tensor([ 97,  24, 168])
8
tensor([ 24, 168, 134])
8
tensor([168, 134,  80])
8
tensor([134,  80,  34])
8
tensor([ 80,  34, 180])
8
tensor([ 34, 180,  12])
8
tensor([180,  12,  40])
8
tensor([12, 40, 27])
8
tensor([40, 27, 44])
8
tensor([ 27,  44, 137])
8
tensor([ 44, 137,  47])
8
tensor([137,  47, 183])
8
tensor([ 47, 183,  41])
8
tensor([183,  41,  25])
8
tensor([ 41,  25, 184])
8
tensor([ 25, 184,  40])
8
tensor([184,  40,  17])
8
tensor([40, 17, 11])
8
tensor([ 17,  11, 143])
8
tensor([ 11, 143, 138])
8
tensor([143, 138,  80])
8
tensor([138,

...

41
tensor([ 76,   3, 146])
41
tensor([  3, 146,  40])
41
tensor([146,  40, 149])
41
tensor([ 40, 149, 118])
41
tensor([149, 118,  85])
41
tensor([118,  85,   5])
41
tensor([85,  5, 96])
41
tensor([  5,  96, 180])
41
tensor([ 96, 180, 158])
41
tensor([180, 158, 104])
41
tensor([158, 104,  10])
41
tensor([104,  10,  96])
41
tensor([ 10,  96, 180])
41
tensor([ 96, 180, 189])
41
tensor([180, 189, 187])
41
tensor([189, 187, 165])
41
tensor([187, 165, 177])
41
tensor([165, 177,  54])
41
tensor([177,  54,  43])
41
tensor([ 54,  43, 166])
41
tensor([ 43, 166,  79])
41
tensor([166,  79,  94])
41
tensor([ 79,  94, 180])
41
tensor([ 94, 180, 192])
41
tensor([180, 192,  49])
41
tensor([192,  49, 117])
41
tensor([ 49, 117, 151])
41
tensor([117, 151, 171])
41
tensor([151, 171, 108])
41
tensor([171, 108, 119])
41
tensor([108, 119, 144])
41
tensor([119, 144,  76])
41
tensor([144,  76, 152])
41
tensor([ 76, 152,  71])
41
tensor([152,  71,  18])
41
tensor([71, 18, 87])
41
tensor([18, 87, 15])
41
tensor(

43
tensor([ 47, 183, 178])
43
tensor([183, 178,  40])
43
tensor([178,  40, 153])
43
tensor([ 40, 153, 132])
43
tensor([153, 132,  86])
43
tensor([132,  86, 117])
43
tensor([ 86, 117,  37])
43
tensor([117,  37,  13])
43
tensor([37, 13, 49])
43
tensor([ 13,  49, 142])
43
tensor([ 49, 142, 193])
43
tensor([142, 193, 102])
43
tensor([193, 102, 181])
43
tensor([102, 181,  40])
43
tensor([181,  40,   1])
43
tensor([40,  1,  4])
43
tensor([ 1,  4, 35])
43
tensor([  4,  35, 180])
43
tensor([ 35, 180,  63])
43
tensor([180,  63,  40])
43
tensor([63, 40, 75])
43
tensor([40, 75, 28])
43
tensor([75, 28, 48])
43
tensor([ 28,  48, 106])
43
tensor([ 48, 106, 133])
43
tensor([106, 133,  96])
43
tensor([133,  96, 180])
43
tensor([ 96, 180, 169])
43
tensor([180, 169,  89])
43
tensor([169,  89,  18])
43
tensor([89, 18, 87])
43
tensor([18, 87, 86])
43
tensor([87, 86, 98])
43
tensor([ 86,  98, 105])
43
tensor([ 98, 105, 139])
43
tensor([105, 139,  70])
43
tensor([139,  70,  51])
43
tensor([70, 51, 86])
43
t

44
tensor([46, 59, 76])
44
tensor([59, 76,  3])
44
tensor([ 76,   3, 146])
44
tensor([  3, 146,  40])
44
tensor([146,  40, 149])
44
tensor([ 40, 149, 118])
44
tensor([149, 118,  85])
44
tensor([118,  85,   5])
44
tensor([85,  5, 96])
44
tensor([  5,  96, 180])
44
tensor([ 96, 180, 158])
44
tensor([180, 158, 104])
44
tensor([158, 104,  10])
44
tensor([104,  10,  96])
44
tensor([ 10,  96, 180])
44
tensor([ 96, 180, 189])
44
tensor([180, 189, 187])
44
tensor([189, 187, 165])
44
tensor([187, 165, 177])
44
tensor([165, 177,  54])
44
tensor([177,  54,  43])
44
tensor([ 54,  43, 166])
44
tensor([ 43, 166,  79])
44
tensor([166,  79,  94])
44
tensor([ 79,  94, 180])
44
tensor([ 94, 180, 192])
44
tensor([180, 192,  49])
44
tensor([192,  49, 117])
44
tensor([ 49, 117, 151])
44
tensor([117, 151, 171])
44
tensor([151, 171, 108])
44
tensor([171, 108, 119])
44
tensor([108, 119, 144])
44
tensor([119, 144,  76])
44
tensor([144,  76, 152])
44
tensor([ 76, 152,  71])
44
tensor([152,  71,  18])
44
tensor(

45
tensor([ 42,  35, 180])
45
tensor([ 35, 180,  36])
45
tensor([180,  36, 107])
45
tensor([ 36, 107,   0])
45
tensor([107,   0, 155])
45
tensor([  0, 155,  40])
45
tensor([155,  40,  68])
45
tensor([40, 68, 23])
45
tensor([ 68,  23, 180])
45
tensor([ 23, 180,   9])
45
tensor([180,   9,  38])
45
tensor([ 9, 38, 53])
45
tensor([ 38,  53, 117])
45
tensor([ 53, 117,  60])
45
tensor([117,  60,  90])
45
tensor([ 60,  90, 129])
45
tensor([ 90, 129,  20])
45
tensor([129,  20,   3])
45
tensor([20,  3, 77])
45
tensor([ 3, 77, 40])
45
tensor([ 77,  40, 130])
45
tensor([ 40, 130,  35])
45
tensor([130,  35, 126])
45
tensor([ 35, 126,  22])
45
tensor([126,  22,  33])
45
tensor([22, 33, 95])
45
tensor([ 33,  95, 172])
45
tensor([ 95, 172, 177])
45
tensor([172, 177,   6])
45
tensor([177,   6,  40])
45
tensor([  6,  40, 164])
1030.260410308838
46
tensor([ 92,  69, 180])
46
tensor([ 69, 180, 179])
46
tensor([180, 179,  80])
46
tensor([179,  80,  66])
46
tensor([ 80,  66, 141])
46
tensor([ 66, 141, 163]

KeyboardInterrupt: 